In [1]:
import numpy as np
import pandas as pd
import scipy
from matplotlib import pyplot as plt
import seaborn as sns
from matplotlib.ticker import MaxNLocator

In [2]:
df_pre = pd.read_csv('./data/pre_study_questionnaire_updated.csv', sep=",")
df = pd.read_csv('./data/post_study_questionnaire_updated.csv', sep=",")
rg_df = pd.read_csv('./data/field_data_field_research_group_updated.csv', sep=",")
recommendation_clicks_df = pd.read_csv('./data/wiski_log_recommendation_clicks_updated.csv', sep=",")
quiz_result_answer_df = pd.read_csv('./data/quiz_node_results_answers_updated.csv', sep=",")
quiz_result_df = pd.read_csv('./data/quiz_node_results_updated.csv', sep=",")

In [3]:
df = df[df['Serienummer']>7]
df_pre = df_pre[df_pre["Serienummer"]>12]
result_df = pd.merge(df,rg_df,how='inner', left_on='UID',right_on='entity_id')
grade_df = df_pre[['UID', 'In welk leerjaar zit je?']]
result_df = pd.merge(result_df,grade_df,how='inner',left_on='UID',right_on='UID')
result_df = result_df.drop_duplicates(subset=['UID']) 
result_df = result_df[result_df['In welk leerjaar zit je?']!=" Anders"]

In [4]:
valid_uids = set(result_df['UID'].unique())

In [5]:
recommendation_clicks_df = recommendation_clicks_df[recommendation_clicks_df["uid"].isin(valid_uids)]
recommendation_clicks_df.columns=recommendation_clicks_df.columns.str.strip()

In [6]:
counts = dict()
valid_uids = list(valid_uids)
for uid in valid_uids:
    uid_df = recommendation_clicks_df[recommendation_clicks_df['uid']==uid]
    sorted_df = uid_df.sort_values(by=["timestamp"], axis=0)

    last_row = sorted_df.iloc[-1]
    last_click = last_row['element']
    count = counts.get(last_click, 0)
    counts[last_click] = count + 1

In [7]:
valid_uids = list(valid_uids)
for uid in valid_uids:
    uid_df = recommendation_clicks_df[recommendation_clicks_df['uid']==uid]
    sorted_df = uid_df.sort_values(by=["timestamp"], axis=0)
    recommendation_clicks_df.drop(sorted_df.iloc[0].name,  inplace=True)
len(recommendation_clicks_df)

435

In [8]:
mapping = dict()
valid_uids = list(valid_uids)
initial_rec_click_df = pd.DataFrame(columns=recommendation_clicks_df.columns)
for uid in valid_uids:
    uid_df = recommendation_clicks_df[recommendation_clicks_df['uid']==uid]
    sorted_df = uid_df.sort_values(by=["timestamp"], axis=0)
    first_five_df = sorted_df.head(5)
    initial_rec_click_df=initial_rec_click_df.append(first_five_df)

In [9]:
buttons = ["1st Rec.", "2nd Rec.", "3rd Rec.", "Not Accept"]

In [10]:
joined_df = pd.merge(quiz_result_df,quiz_result_answer_df,how='right', left_on='result_id',right_on='result_id')

In [11]:
joined_df = joined_df[joined_df["uid"].isin(valid_uids)]

In [12]:
rg_df = rg_df[['entity_id', 'field_research_group_value']]
df = pd.merge(rg_df,initial_rec_click_df,how='right',left_on='entity_id',right_on='uid')
df = df[df["uid"].isin(valid_uids)]

In [13]:
real_df = df[df["field_research_group_value"]==0]
placebo_df = df[df["field_research_group_value"]==1] 
nothing_df = df[df["field_research_group_value"]==2]

Find the proportion of recommendations

In [14]:
real_df.columns

Index(['entity_id', 'field_research_group_value', 'id', 'uid',
       'quiz_result_id', 'quiz_result_answer_id', 'element', 'timestamp',
       'attempts'],
      dtype='object')

In [15]:
real_cut_df = real_df[['uid', 'element']]
placebo_cut_df = placebo_df[['uid', 'element']]
nothing_cut_df = nothing_df[['uid', 'element']]

In [16]:
grouped_multiple = real_cut_df.groupby(['uid', 'element']).agg({'element': ['count']})
grouped_multiple.columns = ['count']
grouped_multiple = grouped_multiple.reset_index()
grouped_multiple_2 = real_df.groupby(['uid']).agg({'element': ['count']})
grouped_multiple_2.columns = ['count_total']
grouped_multiple_2 = grouped_multiple_2.reset_index()
grouped_multiple = pd.merge(grouped_multiple, grouped_multiple_2, how="left", left_on="uid", right_on="uid")
grouped_multiple["proportion"] = grouped_multiple['count']/grouped_multiple['count_total']
grouped_multiple_real = grouped_multiple
grouped_multiple_real_under_5 = grouped_multiple_real[grouped_multiple_real["count_total"]<4]

In [17]:
grouped_multiple = placebo_cut_df.groupby(['uid', 'element']).agg({'element': ['count']})
grouped_multiple.columns = ['count']
grouped_multiple = grouped_multiple.reset_index()
grouped_multiple_2 = placebo_df.groupby(['uid']).agg({'element': ['count']})
grouped_multiple_2.columns = ['count_total']
grouped_multiple_2 = grouped_multiple_2.reset_index()
grouped_multiple = pd.merge(grouped_multiple, grouped_multiple_2, how="left", left_on="uid", right_on="uid")
grouped_multiple["proportion"] = grouped_multiple['count']/grouped_multiple['count_total']
grouped_multiple_placebo = grouped_multiple
grouped_multiple_placebo
grouped_multiple_placebo_under_5 = grouped_multiple_placebo[grouped_multiple_placebo["count_total"]<4]

In [18]:
# new
cut_df = recommendation_clicks_df[['uid', 'element']]

grouped_multiple = cut_df.groupby(['uid', 'element']).agg({'element': ['count']})
grouped_multiple.columns = ['count']
grouped_multiple = grouped_multiple.reset_index()
grouped_multiple_2 = recommendation_clicks_df.groupby(['uid']).agg({'element': ['count']})
grouped_multiple_2.columns = ['count_total']
grouped_multiple_2 = grouped_multiple_2.reset_index()
grouped_multiple = pd.merge(grouped_multiple, grouped_multiple_2, how="left", left_on="uid", right_on="uid")
grouped_multiple["proportion"] = grouped_multiple['count']/grouped_multiple['count_total']
grouped_multiple_all = grouped_multiple
uids_more_than_30 = [1887, 1970, 1895]
grouped_multiple_all[grouped_multiple_all["uid"].isin(uids_more_than_30)]

,uid,element,count,count_total,proportion
2,1887,Button: First Recommended,94,97,0.969072
3,1887,Button: Oefeningenoverzicht,1,97,0.010309
4,1887,Button: Onderwerpenoverzicht,2,97,0.020619
19,1895,Button: First Recommended,26,27,0.962963
20,1895,Button: Oefeningenoverzicht,1,27,0.037037
61,1970,Button: First Recommended,29,31,0.935484
62,1970,Button: Oefeningenoverzicht,2,31,0.064516


In [19]:
grouped_multiple = nothing_cut_df.groupby(['uid', 'element']).agg({'element': ['count']})
grouped_multiple.columns = ['count']
grouped_multiple = grouped_multiple.reset_index()
grouped_multiple_2 = nothing_df.groupby(['uid']).agg({'element': ['count']})
grouped_multiple_2.columns = ['count_total']
grouped_multiple_2 = grouped_multiple_2.reset_index()
grouped_multiple = pd.merge(grouped_multiple, grouped_multiple_2, how="left", left_on="uid", right_on="uid")
grouped_multiple["proportion"] = grouped_multiple['count']/grouped_multiple['count_total']
grouped_multiple_nothing = grouped_multiple
grouped_multiple_nothing
grouped_multiple_nothing_under_5 = grouped_multiple_nothing[grouped_multiple_nothing["count_total"]<4]

In [20]:
grouped_multiple_2 = real_df.groupby(['uid']).agg({'element': ['count']})
grouped_multiple_2.columns = ['count']
grouped_multiple_2 = grouped_multiple_2.reset_index()
print(grouped_multiple_2)

     uid  count
0   1842      1
1   1886      4
2   1891      4
3   1894      5
4   1895      5
5   1910      5
6   1912      4
7   1930      4
8   1932      4
9   1970      5
10  1973      4
11  1976      5


In [21]:
print(grouped_multiple[grouped_multiple['element']!="Button: Oefeningenoverzicht"])
grouped_multiple = grouped_multiple_real
grouped_multiple_accept_real = grouped_multiple[grouped_multiple['element']!="Button: Oefeningenoverzicht"]
grouped_multiple_accept_real = grouped_multiple_accept_real.groupby(['uid']).agg({'proportion': ['sum']})
grouped_multiple_accept_real.columns = ['acceptance']
grouped_multiple_accept_real = grouped_multiple_accept_real.reset_index()
grouped_multiple_accept_real["Research Group"] = "IRE"
grouped_multiple_accept_real

     uid                    element  count  count_total  proportion
0   1888  Button: First Recommended      5            5    1.000000
1   1889  Button: First Recommended      3            5    0.600000
3   1892  Button: First Recommended      3            3    1.000000
4   1893  Button: First Recommended      4            5    0.800000
6   1899  Button: First Recommended      2            5    0.400000
9   1921  Button: Third Recommended      2            3    0.666667
10  1941  Button: First Recommended      4            5    0.800000
12  1944  Button: First Recommended      4            4    1.000000
13  1948  Button: First Recommended      4            5    0.800000
15  1949  Button: First Recommended      4            5    0.800000
17  1950  Button: First Recommended      3            4    0.750000
19  1967  Button: First Recommended      1            5    0.200000
21  1969  Button: First Recommended      4            5    0.800000


,uid,acceptance,Research Group
0,1842,1.0,IRE
1,1886,1.0,IRE
2,1891,0.5,IRE
3,1894,1.0,IRE
4,1895,1.0,IRE
5,1930,1.0,IRE
6,1932,1.0,IRE
7,1970,0.8,IRE
8,1973,1.0,IRE
9,1976,1.0,IRE


In [22]:
grouped_multiple = grouped_multiple_placebo
grouped_multiple_accept_placebo = grouped_multiple[grouped_multiple['element']!="Button: Oefeningenoverzicht"]
grouped_multiple_accept_placebo = grouped_multiple_accept_placebo.groupby(['uid']).agg({'proportion': ['sum']})
grouped_multiple_accept_placebo.columns = ['acceptance']
grouped_multiple_accept_placebo = grouped_multiple_accept_placebo.reset_index()
grouped_multiple_accept_placebo["Research Group"] = "IPE"
grouped_multiple_accept_placebo

,uid,acceptance,Research Group
0,1887,1.0,IPE
1,1896,0.8,IPE
2,1911,0.5,IPE
3,1914,1.0,IPE
4,1938,1.0,IPE
5,1939,1.0,IPE
6,1947,0.8,IPE
7,1956,0.6,IPE
8,1958,1.0,IPE
9,1962,1.0,IPE


In [23]:
grouped_multiple = grouped_multiple_nothing
grouped_multiple_accept_nothing = grouped_multiple[grouped_multiple['element']!="Button: Oefeningenoverzicht"]
grouped_multiple_accept_nothing = grouped_multiple_accept_nothing.groupby(['uid']).agg({'proportion': ['sum']})
grouped_multiple_accept_nothing.columns = ['acceptance']
grouped_multiple_accept_nothing = grouped_multiple_accept_nothing.reset_index()
grouped_multiple_accept_nothing["Research Group"] = "INE"
grouped_multiple_accept_nothing

,uid,acceptance,Research Group
0,1888,1.000000,INE
1,1889,0.600000,INE
2,1892,1.000000,INE
3,1893,0.800000,INE
4,1899,0.400000,INE
5,1921,0.666667,INE
6,1941,0.800000,INE
7,1944,1.000000,INE
8,1948,0.800000,INE
9,1949,0.800000,INE


In [24]:
frames = [grouped_multiple_accept_real, grouped_multiple_accept_placebo, grouped_multiple_accept_nothing]
acceptance_df = pd.concat(frames)

In [25]:
q1 = '1. Wiski is zoals een expert (bv. een leerkracht) in wiskunde-oefeningen aanraden.'
q2 = '2. Wiski heeft de expertise (kennis) om mijn wiskundeniveau te kunnen inschatten.'
q3 = '3. Wiski kan mijn wiskundeniveau inschatten.'
q4 = '4. Wiski begrijpt de moeilijkheidsgraad van wiskunde-oefeningen goed.'
q5 = '5. Wiski houdt rekening met mijn wiskundeniveau om oefeningen aan te raden.'
q6 = '6. Wiski zet op de eerste plaats dat ik vorderingen maak in wiskunde.'
q7 = '7. Wanneer Wiski oefeningen aanraadt, doet Wiski dat zodat ik vorderingen maak in wiskunde.'
q8 = '8. Wiski wilt mijn wiskundeniveau goed inschatten.'
q9 = '9. Wiski raadt oefeningen op een zo correct mogelijke manier aan.'
q10 = '10. Wiski is eerlijk.'
q11 = '11. Wiski maakt oprechte aanbevelingen.'
q12 = '12. Ik vertrouw Wiski om mij wiskunde-oefeningen aan te raden.'
q13 = '13. Als ik nog eens online wiskunde-oefeningen maak, dan kies ik voor Wiski.'
q14 = '14. Als ik nog eens wiskunde-oefeningen aangeraden wil krijgen, dan kies ik voor Wiski.'
q15 = '15. Ik vind dat Wiski genoeg uitleg geeft over waarom een oefening aangeraden is.'
q16 = '16. Wanneer ik Wiski gebruik, wil ik GEEN uitleg over waarom een oefening wordt aangeraden.'
q17 = '17. Ik vind uitleg krijgen over waarom een oefening wordt aangeraden belangrijker dan waarom een film wordt aangeraden.'
q18 = '18. Ik ben NIET blij met het niveau van de oefeningen die Wiski aanraadde.'
q19 = '19. In het algemeen vind ik het belangrijk om uitleg te krijgen wanneer iets (oefening/film/product/...) wordt aangeraden.'


def map_likert_to_int(key):
    if key == "Helemaal oneens":
        return 1
    elif key == "Oneens":
        return 2
    elif key == "Eerder oneens":
        return 3
    elif key == "Neutraal":
        return 4
    elif key == "Eerder eens":
        return 5
    elif key == "Eens":
        return 6
    elif key == "Helemaal eens":
        return 7
def dic_scores_competence(df):
    scores = dict()
    for i in range(len(df)):
        row = df.iloc[i]
        score = 0
        score += int(map_likert_to_int(row[q1].strip()))
        score += int(map_likert_to_int(row[q2].strip()))
        score += int(map_likert_to_int(row[q3].strip()))
        score += int(map_likert_to_int(row[q4].strip()))
        score += int(map_likert_to_int(row[q5].strip()))
        scores[row['UID']] = score/5
    return scores
def dic_scores_benevolence(df):
    scores = dict()
    for i in range(len(df)):
        row = df.iloc[i]
        score = 0
        score += int(map_likert_to_int(row[q6].strip()))
        score += int(map_likert_to_int(row[q7].strip()))
        score += int(map_likert_to_int(row[q8].strip()))
        scores[row['UID']] = score/3
    return scores
def dic_scores_integrity(df):
    scores = dict()
    for i in range(len(df)):
        row = df.iloc[i]
        score = 0
        score += int(map_likert_to_int(row[q9].strip()))
        score += int(map_likert_to_int(row[q10].strip()))
        score += int(map_likert_to_int(row[q11].strip()))
        scores[row['UID']] = score/3
    return scores       
def dic_scores_trusting_beliefs(competence, benevolence, integrity):
    scores = dict()
    for pair in competence.items():
        uid = pair[0]
        scores[uid] = competence[uid]+benevolence[uid]+integrity[uid]
    return scores
def dic_scores_perceived_transparency(df):
    scores = dict()
    for i in range(len(df)):
        row = df.iloc[i]
        score = 0
        score += int(map_likert_to_int(row[q15].strip()))
        scores[row['UID']] = score
    return scores
def dic_scores_intention_to_return(df):
    scores = dict()
    for i in range(len(df)):
        row = df.iloc[i]
        score = 0
        score += int(map_likert_to_int(row[q13].strip()))
        score += int(map_likert_to_int(row[q14].strip()))
        scores[row['UID']] = score/2
    return scores
def dic_scores_single_q(df, q_number):
    scores = dict()
    for i in range(len(df)):
        row = df.iloc[i]
        score = 0
        score += int(map_likert_to_int(row[q_number].strip()))
        scores[row['UID']] = score
    return scores
def dic_scores_trust_sho(trusting_beliefs, intention_to_return, perceived_transparency):
    scores = dict()
    for pair in trusting_beliefs.items():
        uid = pair[0]
        scores[uid] = trusting_beliefs[uid] + intention_to_return[uid] + perceived_transparency[uid]
    return scores
def dic_scores_trust_sho_reweighted(trusting_beliefs, intention_to_return, perceived_transparency):
    scores = dict()
    for pair in trusting_beliefs.items():
        uid = pair[0]
        scores[uid] = trusting_beliefs[uid]/3 + intention_to_return[uid] + perceived_transparency[uid]
    return scores

In [26]:
result_df = pd.merge(result_df,rg_df,how='inner', left_on='UID',right_on='entity_id')
result_df = result_df.rename(columns={'field_research_group_value_x': 'field_research_group_value'})

In [27]:

result_real_df = result_df[result_df["field_research_group_value"]==0]
result_placebo_df = result_df[result_df["field_research_group_value"]==1]
result_nothing_df = result_df[result_df["field_research_group_value"]==2]

In [28]:
print(len(result_df))
print(len(result_real_df))
print(len(result_placebo_df))
print(len(result_nothing_df))

37
12
12
13


In [29]:
competence_scores_real = dic_scores_competence(result_real_df)
competence_scores_placebo = dic_scores_competence(result_placebo_df)
competence_scores_nothing = dic_scores_competence(result_nothing_df)
competence_scores = dict(competence_scores_real)
competence_scores.update(competence_scores_placebo)
competence_scores.update(competence_scores_nothing)
competence_df = pd.DataFrame.from_dict(competence_scores, orient="index", columns=["Competence"]).reset_index()
competence_df.columns = ["uid", "Competence"]

benevolence_scores_real = dic_scores_benevolence(result_real_df)
benevolence_scores_placebo = dic_scores_benevolence(result_placebo_df)
benevolence_scores_nothing = dic_scores_benevolence(result_nothing_df)
benevolence_scores = dict(benevolence_scores_real)
benevolence_scores.update(benevolence_scores_placebo)
benevolence_scores.update(benevolence_scores_nothing)
benevolence_df = pd.DataFrame.from_dict(benevolence_scores, orient="index").reset_index()
benevolence_df.columns = ["uid", "Benevolence"]


integrity_scores_real = dic_scores_integrity(result_real_df)
integrity_scores_placebo = dic_scores_integrity(result_placebo_df)
integrity_scores_nothing = dic_scores_integrity(result_nothing_df)
integrity_scores = dict(integrity_scores_real)
integrity_scores.update(integrity_scores_placebo)
integrity_scores.update(integrity_scores_nothing)
integrity_df = pd.DataFrame.from_dict(integrity_scores, orient="index").reset_index()
integrity_df.columns = ["uid", "Integrity"]



trusting_beliefs_scores_real = dic_scores_trusting_beliefs(competence_scores_real, benevolence_scores_real, integrity_scores_real)
trusting_beliefs_scores_placebo = dic_scores_trusting_beliefs(competence_scores_placebo, benevolence_scores_placebo, integrity_scores_placebo)
trusting_beliefs_scores_nothing = dic_scores_trusting_beliefs(competence_scores_nothing, benevolence_scores_nothing, integrity_scores_nothing)
trusting_beliefs_scores = dict(trusting_beliefs_scores_real)
trusting_beliefs_scores.update(trusting_beliefs_scores_placebo)
trusting_beliefs_scores.update(trusting_beliefs_scores_nothing)
trusting_beliefs_df = pd.DataFrame.from_dict(trusting_beliefs_scores, orient="index").reset_index()
trusting_beliefs_df.columns = ["uid", "Trusting Beliefs"]



itr_scores_real = dic_scores_intention_to_return(result_real_df)
itr_scores_placebo = dic_scores_intention_to_return(result_placebo_df)
itr_scores_nothing = dic_scores_intention_to_return(result_nothing_df)
itr_scores = dict(itr_scores_real)
itr_scores.update(itr_scores_placebo)
itr_scores.update(itr_scores_nothing)
itr_df = pd.DataFrame.from_dict(itr_scores, orient="index").reset_index()
itr_df.columns = ["uid", "Intention to Return"]

pt_scores_real = dic_scores_perceived_transparency(result_real_df)
pt_scores_placebo = dic_scores_perceived_transparency(result_placebo_df)
pt_scores_nothing = dic_scores_perceived_transparency(result_nothing_df)
pt_scores = dict(pt_scores_real)
pt_scores.update(pt_scores_placebo)
pt_scores.update(pt_scores_nothing)
pt_df = pd.DataFrame.from_dict(pt_scores, orient="index").reset_index()
pt_df.columns = ["uid", "Perceived Transparency"]


explicit_trust_scores_real = dic_scores_single_q(result_real_df, q12)
explicit_trust_scores_placebo = dic_scores_single_q(result_placebo_df, q12)
explicit_trust_scores_nothing = dic_scores_single_q(result_nothing_df, q12)
explicit_trust_scores = dict(explicit_trust_scores_real)
explicit_trust_scores.update(explicit_trust_scores_placebo)
explicit_trust_scores.update(explicit_trust_scores_nothing)
explicit_trust_df = pd.DataFrame.from_dict(explicit_trust_scores, orient="index").reset_index()
explicit_trust_df.columns = ["uid", "1D Trust"]

trust_sho_scores_real = dic_scores_trust_sho(trusting_beliefs_scores_real, itr_scores_real, pt_scores_real)
trust_sho_scores_placebo = dic_scores_trust_sho(trusting_beliefs_scores_placebo, itr_scores_placebo, pt_scores_placebo)
trust_sho_scores_nothing = dic_scores_trust_sho(trusting_beliefs_scores_nothing, itr_scores_nothing, pt_scores_nothing)
trust_sho_scores = dict(trust_sho_scores_real)
trust_sho_scores.update(trust_sho_scores_placebo)
trust_sho_scores.update(trust_sho_scores_nothing)
trust_sho_df = pd.DataFrame.from_dict(trust_sho_scores, orient="index").reset_index()
trust_sho_df.columns = ["uid", "MD Trust"]


rw_trust_sho_scores_real = dic_scores_trust_sho_reweighted(trusting_beliefs_scores_real, itr_scores_real, pt_scores_real)
rw_trust_sho_scores_placebo = dic_scores_trust_sho_reweighted(trusting_beliefs_scores_placebo, itr_scores_placebo, pt_scores_placebo)
rw_trust_sho_scores_nothing = dic_scores_trust_sho_reweighted(trusting_beliefs_scores_nothing, itr_scores_nothing, pt_scores_nothing)
rw_trust_sho_scores = dict(rw_trust_sho_scores_real)
rw_trust_sho_scores.update(rw_trust_sho_scores_placebo)
rw_trust_sho_scores.update(rw_trust_sho_scores_nothing)
rw_trust_sho_df = pd.DataFrame.from_dict(rw_trust_sho_scores, orient="index").reset_index()
rw_trust_sho_df.columns = ["uid", "MD Trust (RW)"]

In [30]:
response_df = pd.merge(competence_df, benevolence_df, how="inner", left_on="uid", right_on="uid")
response_df = pd.merge(response_df, integrity_df, how="inner", left_on="uid", right_on="uid")
response_df = pd.merge(response_df, trusting_beliefs_df, how="inner", left_on="uid", right_on="uid")
response_df = pd.merge(response_df, itr_df, how="inner", left_on="uid", right_on="uid")
response_df = pd.merge(response_df, pt_df, how="inner", left_on="uid", right_on="uid")
response_df = pd.merge(response_df, explicit_trust_df, how="inner", left_on="uid", right_on="uid")
response_df = pd.merge(response_df, trust_sho_df, how="inner", left_on="uid", right_on="uid")
response_df = pd.merge(response_df, rw_trust_sho_df, how="inner", left_on="uid", right_on="uid")

In [31]:
response_df

,uid,Competence,Benevolence,Integrity,Trusting Beliefs,Intention to Return,Perceived Transparency,1D Trust,MD Trust,MD Trust (RW)
0,1886,6.2,6.333333,6.666667,19.200000,5.0,7,6,31.200000,18.400000
1,1891,6.2,6.666667,5.666667,18.533333,6.0,4,6,28.533333,16.177778
2,1894,5.6,6.333333,5.666667,17.600000,6.0,3,5,26.600000,14.866667
3,1895,6.4,7.000000,7.000000,20.400000,7.0,7,7,34.400000,20.800000
4,1912,4.4,5.666667,5.333333,15.400000,4.0,3,5,22.400000,12.133333
5,1910,6.0,6.000000,5.666667,17.666667,6.0,5,6,28.666667,16.888889
6,1932,6.2,6.000000,6.666667,18.866667,6.0,7,7,31.866667,19.288889
7,1842,6.2,5.333333,6.666667,18.200000,5.0,7,7,30.200000,18.066667
8,1930,6.2,5.666667,6.666667,18.533333,6.5,7,7,32.033333,19.677778
9,1970,5.4,6.000000,6.000000,17.400000,6.0,6,6,29.400000,17.800000


In [32]:
likert_response_and_acceptance_df = pd.merge(response_df, acceptance_df, how="inner", left_on="uid", right_on="uid")
likert_response_and_acceptance_df

,uid,Competence,Benevolence,Integrity,Trusting Beliefs,Intention to Return,Perceived Transparency,1D Trust,MD Trust,MD Trust (RW),acceptance,Research Group
0,1886,6.2,6.333333,6.666667,19.200000,5.0,7,6,31.200000,18.400000,1.000000,IRE
1,1891,6.2,6.666667,5.666667,18.533333,6.0,4,6,28.533333,16.177778,0.500000,IRE
2,1894,5.6,6.333333,5.666667,17.600000,6.0,3,5,26.600000,14.866667,1.000000,IRE
3,1895,6.4,7.000000,7.000000,20.400000,7.0,7,7,34.400000,20.800000,1.000000,IRE
4,1932,6.2,6.000000,6.666667,18.866667,6.0,7,7,31.866667,19.288889,1.000000,IRE
5,1842,6.2,5.333333,6.666667,18.200000,5.0,7,7,30.200000,18.066667,1.000000,IRE
6,1930,6.2,5.666667,6.666667,18.533333,6.5,7,7,32.033333,19.677778,1.000000,IRE
7,1970,5.4,6.000000,6.000000,17.400000,6.0,6,6,29.400000,17.800000,0.800000,IRE
8,1973,6.0,6.000000,6.666667,18.666667,6.0,5,6,29.666667,17.222222,1.000000,IRE
9,1976,3.0,5.000000,4.666667,12.666667,5.0,4,3,21.666667,13.222222,1.000000,IRE


In [33]:
likert_response_and_acceptance_df.columns = ["uid","Competence", "Benevolence", "Integrity", "Trusting Beliefs", "Intention to Return", "Perceived Transparency", "1D Trust", "MD Trust", "MD Trust (RW)", "Acceptance", "Research Group"]

In [34]:
likert_response_and_acceptance_df[["Competence", "Benevolence", "Integrity", "Trusting Beliefs", "Intention to Return", "Perceived Transparency", "1D Trust", "MD Trust", "MD Trust (RW)"]].corr()

,Competence,Benevolence,Integrity,Trusting Beliefs,Intention to Return,Perceived Transparency,1D Trust,MD Trust,MD Trust (RW)
Competence,1.000000,0.646340,0.794712,0.933893,0.679337,0.137921,0.845466,0.832921,0.669334
Benevolence,0.646340,1.000000,0.563403,0.810573,0.627611,0.124466,0.556565,0.734871,0.599296
Integrity,0.794712,0.563403,1.000000,0.900456,0.611051,0.127345,0.836790,0.789226,0.624044
Trusting Beliefs,0.933893,0.810573,0.900456,1.000000,0.722700,0.147042,0.856491,0.890374,0.714397
Intention to Return,0.679337,0.627611,0.611051,0.722700,1.000000,0.045475,0.689548,0.740583,0.665564
Perceived Transparency,0.137921,0.124466,0.127345,0.147042,0.045475,1.000000,0.178502,0.545166,0.741786
1D Trust,0.845466,0.556565,0.836790,0.856491,0.689548,0.178502,1.000000,0.803989,0.675499
MD Trust,0.832921,0.734871,0.789226,0.890374,0.740583,0.545166,0.803989,1.000000,0.954623
MD Trust (RW),0.669334,0.599296,0.624044,0.714397,0.665564,0.741786,0.675499,0.954623,1.000000


In [35]:
correct_grade_df = pd.read_csv("correct_grade_parsed.csv")

In [36]:
correct_grade_response_df = pd.merge(response_df, correct_grade_df, how="inner", left_on='uid', right_on="UID")

In [37]:
correct_grade_response_df

,uid,Competence,Benevolence,Integrity,Trusting Beliefs,Intention to Return,Perceived Transparency,1D Trust,MD Trust,MD Trust (RW),...,UID,Wat is je leeftijd?,Wat is je geslacht?,In welk leerjaar zit je?,Hoeveel uur wiskunde krijg je per week?,"Hoe goed ben je, volgens jezelf, in wiskunde?","Hoe goed ben je, volgens jezelf, met het gebruik van computers?","Gebruikte je al eens een andere website om online oefeningen te maken? (bijvoorbeeld voor wiskunde, Nederlands ...)","Gebruik je websites waar producten/films/... worden aangeraden? (zoals Netflix, Amazon, Bol, ...)",Ik bezoek Wiski niet met Safari of Firefox.
0,1886,6.2,6.333333,6.666667,19.200000,5.0,7,6,31.200000,18.400000,...,1886,16,Meisje,4e middelbaar,5,4,4,Ja,Ja,Ja
1,1891,6.2,6.666667,5.666667,18.533333,6.0,4,6,28.533333,16.177778,...,1891,15,Meisje,6e middelbaar,5,4,3,Ja,Nee,Ja
2,1894,5.6,6.333333,5.666667,17.600000,6.0,3,5,26.600000,14.866667,...,1894,15,Meisje,4e middelbaar,5,3,4,Ja,Ja,Ja
3,1895,6.4,7.000000,7.000000,20.400000,7.0,7,7,34.400000,20.800000,...,1895,15,Meisje,4e middelbaar,5,4,2,Nee,Ja,Ja
4,1912,4.4,5.666667,5.333333,15.400000,4.0,3,5,22.400000,12.133333,...,1912,16,Meisje,4e middelbaar,5,4,4,Ja,Ja,Ja
5,1910,6.0,6.000000,5.666667,17.666667,6.0,5,6,28.666667,16.888889,...,1910,16,Meisje,4e middelbaar,5,3,4,Ja,Ja,Ja
6,1932,6.2,6.000000,6.666667,18.866667,6.0,7,7,31.866667,19.288889,...,1932,15,Meisje,4e middelbaar,5,3,3,Nee,Ja,Ja
7,1842,6.2,5.333333,6.666667,18.200000,5.0,7,7,30.200000,18.066667,...,1842,16,Jongen,5e middelbaar,6,4,5,Ja,Ja,Ja
8,1930,6.2,5.666667,6.666667,18.533333,6.5,7,7,32.033333,19.677778,...,1930,17,Meisje,5e middelbaar,6,4,3,Ja,Ja,Ja
9,1970,5.4,6.000000,6.000000,17.400000,6.0,6,6,29.400000,17.800000,...,1970,18,Jongen,6e middelbaar,2,3,4,Nee,Ja,Ja


In [38]:
correct_grade_response_df.columns

Index(['uid', 'Competence', 'Benevolence', 'Integrity', 'Trusting Beliefs',
       'Intention to Return', 'Perceived Transparency', '1D Trust', 'MD Trust',
       'MD Trust (RW)', 'Unnamed: 0', 'UID', 'Wat is je leeftijd?',
       'Wat is je geslacht?', 'In welk leerjaar zit je?',
       'Hoeveel uur wiskunde krijg je per week?',
       'Hoe goed ben je, volgens jezelf, in wiskunde?',
       'Hoe goed ben je, volgens jezelf, met het gebruik van computers?',
       'Gebruikte je al eens een andere website om online oefeningen te maken? (bijvoorbeeld voor wiskunde, Nederlands ...)',
       'Gebruik je websites waar producten/films/... worden aangeraden? (zoals Netflix, Amazon, Bol, ...)',
       'Ik bezoek Wiski niet met Safari of Firefox.'],
      dtype='object')

In [39]:
q16_scores_real = dic_scores_single_q(result_real_df, q16)
q16_scores_placebo = dic_scores_single_q(result_placebo_df, q16)
q16_scores_nothing = dic_scores_single_q(result_nothing_df, q16)
q16_scores = dict(q16_scores_real)
q16_scores.update(q16_scores_placebo)
q16_scores.update(q16_scores_nothing)
q16_df = pd.DataFrame.from_dict(q16_scores, orient="index").reset_index()
q16_df.columns = ["uid", "Q16"]


q17_scores_real = dic_scores_single_q(result_real_df, q17)
q17_scores_placebo = dic_scores_single_q(result_placebo_df, q17)
q17_scores_nothing = dic_scores_single_q(result_nothing_df, q17)
q17_scores = dict(q17_scores_real)
q17_scores.update(q17_scores_placebo)
q17_scores.update(q17_scores_nothing)
q17_df = pd.DataFrame.from_dict(q17_scores, orient="index").reset_index()
q17_df.columns = ["uid", "Q17"]


q18_scores_real = dic_scores_single_q(result_real_df, q18)
q18_scores_placebo = dic_scores_single_q(result_placebo_df, q18)
q18_scores_nothing = dic_scores_single_q(result_nothing_df, q18)
q18_scores = [q18_scores_real, q18_scores_placebo, q18_scores_nothing]
q18_scores = dict(q18_scores_real)
q18_scores.update(q18_scores_placebo)
q18_scores.update(q18_scores_nothing)
q18_df = pd.DataFrame.from_dict(q18_scores, orient="index").reset_index()
q18_df.columns = ["uid", "Q18"]

q19_scores_real = dic_scores_single_q(result_real_df, q19)
q19_scores_placebo = dic_scores_single_q(result_placebo_df, q19)
q19_scores_nothing = dic_scores_single_q(result_nothing_df, q19)
q19_scores = [q19_scores_real, q19_scores_placebo, q19_scores_nothing]
q19_scores = dict(q19_scores_real)
q19_scores.update(q19_scores_placebo)
q19_scores.update(q19_scores_nothing)
q19_df = pd.DataFrame.from_dict(q19_scores, orient="index").reset_index()
q19_df.columns = ["uid", "Q19"]

In [40]:
response_df = pd.merge(response_df, q16_df, how="inner", left_on="uid", right_on="uid")
response_df = pd.merge(response_df, q17_df, how="inner", left_on="uid", right_on="uid")
response_df = pd.merge(response_df, q18_df, how="inner", left_on="uid", right_on="uid")
response_df = pd.merge(response_df, q19_df, how="inner", left_on="uid", right_on="uid")

In [41]:
correct_grade_response_df.to_csv("correct_grade_response_df.csv")

In [42]:
likert_response_and_acceptance_df.to_csv("likert_response_and_acceptance_df.csv")